![ine-divider](https://user-images.githubusercontent.com/7065401/92672068-398e8080-f2ee-11ea-82d6-ad53f7feb5c0.png)
<hr>

### Secure RESTful APIs using Python

# Token-based authentication project

This project asks you to manage token-based authentication for agent-verified connections.

![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)

## Part 1

**Managing multiple logins**

In the lesson, we showed how a client might create an authenticated identity with a server, and use tokens to validate additional service requests.  The server `project.py` is very similar to the one shown in the lesson.  It varies only in adding a few additional authorized users and in personalizing the response so that each user receives something identifiably distinct from the `/get-info` route.

The `project.py` server runs on port 5014, and uses the same `pubkey.pem` and `private.pem` used in the lesson itself for TLS encryption of the connection.

For this task, you wish to have a client (or other micro-service) that manages multiple collections of credentials to make requests specific to varying users.  Write code to login as each of "Diane" and "Eshana" once, and make requests to the `/get-info` route as each of them several times without requiring a new login.  Feel free examine the code of `project.py`, as well as launching it by running it at a terminal.

**A possible solution**

Once the structure of the lesson is understood, it is clear that maintaining multiple credential sets is not much harder than maintaining a single one.  An obvious approach is simply to maintain a dictionary mapping user to their relevant credential set.

In [5]:
import requests
credentials = dict()

requests.packages.urllib3.disable_warnings()

for user, pwd in [('Diane', 'diane_pw'), ('Eshana', 'eshana_pw')]:
    resp = requests.post("https://localhost:5014/login", verify='pubkey.pem',
                         data=dict(username=user, password=pwd))
    if resp.status_code == 200:
        credentials[user] = resp.cookies

As an example of multi-user credentials, let's make a validated request for each user twice, in an interleaved way.

In [12]:
import json
for _ in range(2):
    for user in ('Diane', 'Eshana'):
        resp = requests.get("https://localhost:5014/get-info", 
                             verify='pubkey.pem', cookies=credentials[user])
        if resp.status_code == 200:
            print(json.loads(resp.text))

{'Diane favorite number': 84}
{'Eshana favorite number': 58}
{'Diane favorite number': 40}
{'Eshana favorite number': 4}


![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)

## Part 2

**Limiting token use**

We would like to write an additional route called `/limited-info` that behaves very similarly to `get-info`. However, unlike `/get-info` which allows authentication tokens to have unlimited use, `/limited-info` will expire tokens after 3 uses.  After that, a new login will be required for use of the service.

For example, we would expect to see behavior like this.  You should write a new server that behaves in this manner.

```python
resp = requests.post("https://localhost:5015/login", verify='pubkey.pem',
                         data=dict(username='Carlos', password='carlos_pw'))
cookies = resp.cookies
for _ in range(5):
    resp = requests.get("https://localhost:5015/limited-info", 
                        verify='pubkey.pem', cookies=cookies)
    if resp.status_code == 200:
        print(json.loads(resp.text))
    else:
        print(resp.status_code, resp.text)
```
```
{'Carlos favorite number': 62}
{'Carlos favorite number': 4}
{'Carlos favorite number': 8}
403 Carlos denied access to resource
403 Carlos denied access to resource
```

Note specifically, that once denied once, the user continues to lack access until they login again (which may or not be permitted under the service terms).

**A possible solution**

As with the first task, understanding the goal is most of the work.  We simply have to keep track on the server of how often a given route has been used by a given user.  For example:

```python
# Global dictionaries
auth_tokens = dict()
usage_count = dict()

# `/login` route defined here ...

@app.route('/limited-info')
def limited_info():
    username = request.cookies.get('app1-username')
    token = request.cookies.get('app1-token', object())
    if token != auth_tokens.get(username):
        return make_response(f"{username} denied access to resource", 403)
    uses = usage_count.get(username, 0)
    if uses >= 3:
        del auth_tokens[username]
        del usage_count[username]
        return make_response(f"{username} denied access to resource", 403)
    else:
        usage_count[username] = uses+1
    return jsonify({f"{username} favorite number": randint(1, 100)})
```

![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)